# Table 1 (Secondary transport population, all ages)

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/rega_data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/rega_data/rega_physician_list_09102025.xlsx'
meta_medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/medreg_extraction/joined_final_complete_extractions_20251008_221735.xlsx'
restrict_to_secondary = True

In [ ]:
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)
meta_medic_df = pd.read_excel(meta_medic_data_path)

medic_df['full_name'] = medic_df['Mitglieder mit Einsatzfunktion'].str.replace(' (Flugarzt/Flugärztin)', '')
medic_df.drop_duplicates(subset=['Mitglieder mit Einsatzfunktion'], inplace=True)
medic_df = medic_df.merge(meta_medic_df, how='left', on='full_name')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

duplicates = data_df[data_df['SNZ Ereignis Nr. '].duplicated()]['SNZ Ereignis Nr. ']
print(f'Duplicates found: {duplicates.nunique()}')
data_df = data_df.drop_duplicates(subset=['SNZ Ereignis Nr. '])

n_vas_under4 = data_df[data_df['VAS_on_scene'] <= 3].shape[0]
print(f'Excluded {n_vas_under4} patients with VAS <= 3')
data_df = data_df[data_df['VAS_on_scene'] > 3]

n_missing_arrival = data_df['VAS_on_arrival'].isna().sum()
print(f'Excluded {n_missing_arrival} patients with missing VAS_on_arrival')
data_df = data_df.dropna(subset=['VAS_on_arrival'])

if restrict_to_secondary:
    n_primary = data_df[data_df['Einsatzart'] != 'Sekundär'].shape[0]
    print(f'Excluded {n_primary} primary transport patients')
    data_df = data_df[data_df['Einsatzart'] == 'Sekundär']

In [ ]:
def get_categorical_str(df, column_name, category, total):
    count = df[df[column_name] == category].shape[0]
    return f'{count} ({count/total:.1%})'

def get_continuous_str(df, column_name, total):
    median = df[column_name].median()
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    return f'{median:.1f} [{q1:.1f} - {q3:.1f}]'

def get_multi_label_counts(data_df, multi_label_column):
    data_df[multi_label_column] = data_df[multi_label_column].replace(999, pd.NA)
    label_counter = {}
    for _, row in data_df.iterrows():
        labels = [label.strip() for label in re.split('; |, ', str(row[multi_label_column]))]
        for label in labels:
            if label == 'nan' or label == '<NA>':
                continue
            if label not in label_counter:
                label_counter[label] = 1
            else:
                label_counter[label] += 1
    sorted_label_counter = dict(sorted(label_counter.items(), key=lambda item: item[1], reverse=True))
    return sorted_label_counter

In [ ]:
def table1(df):
    pop_df = pd.DataFrame()
    str_df = pd.DataFrame()

    n_patients = len(df)
    pop_df['n_patients'] = [n_patients]
    str_df['n_patients'] = [n_patients]

    for naca_level in [2, 3, 4, 5]:
        str_df[f'NACA {naca_level}'] = get_categorical_str(df, 'NACA (nummerisch)', naca_level, n_patients)

    str_df['GCS 13'] = get_categorical_str(df, 'GCS', 13, n_patients)
    str_df['GCS 14'] = get_categorical_str(df, 'GCS', 14, n_patients)
    str_df['GCS 15'] = get_categorical_str(df, 'GCS', 15, n_patients)

    str_df['age'] = get_continuous_str(df, 'Alter ', n_patients)
    str_df['patient_sex_male'] = get_categorical_str(df, 'Geschlecht', 'Männlich', n_patients)

    str_df['physician_sex_male'] = get_categorical_str(df, 'physician_sex', 'm', n_patients)
    df['event_year'] = pd.to_datetime(df['Ereignisdatum'], format='%d.%m.%Y').dt.year
    df['physician_age'] = df['event_year'] - df['year_of_birth']
    df['physician_licence_year'] = df['licence_date'].apply(lambda x: str(x).split('.')[-1] if '.' in str(x) else str(x))
    df['phyisician_experience_years'] = df['event_year'] - pd.to_numeric(df['physician_licence_year'], errors='coerce')
    str_df['physician_age'] = get_continuous_str(df, 'physician_age', n_patients)
    str_df['physician_experience_years'] = get_continuous_str(df, 'phyisician_experience_years', n_patients)

    df['physician_anesthesiologist'] = df['specialist_qualifications'].str.contains('Anaesthesiology', na=False)
    str_df['physician_anesthesiologist'] = get_categorical_str(df, 'physician_anesthesiologist', True, n_patients)
    df['physician_intensivist'] = df['specialist_qualifications'].str.contains('Intensive care medicine', na=False)
    str_df['physician_intensivist'] = get_categorical_str(df, 'physician_intensivist', True, n_patients)
    df['physician_internist'] = df['specialist_qualifications'].str.contains('General Internal Medicine|General medical practitioner', na=False)
    str_df['physician_internist'] = get_categorical_str(df, 'physician_internist', True, n_patients)

    extraction_methods = get_multi_label_counts(df, 'Bergungen')
    n_whinch_extractions = sum(v for k, v in extraction_methods.items() if 'Winde' in k)
    str_df['n_whinch_extractions'] = f'{n_whinch_extractions} ({n_whinch_extractions/n_patients:.1%})'

    df['mission_duration'] = (pd.to_datetime(df['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - pd.to_datetime(df['Erstbefund'], format='%d.%m.%Y %H:%M:%S')).dt.total_seconds() / 60
    str_df['mission_duration'] = get_continuous_str(df, 'mission_duration', n_patients)

    str_df['night_time_mission'] = get_categorical_str(df, 'Tag oder Nacht', 'Nacht', n_patients)

    n_medical = get_multi_label_counts(df, 'Einteilung (reduziert)').get('Krankheit', 0)
    str_df['medical'] = f'{n_medical} ({n_medical/n_patients:.1%})'

    str_df['VAS_on_scene'] = get_continuous_str(df, 'VAS_on_scene', n_patients)
    str_df['VAS_at_hospital_admission'] = get_continuous_str(df, 'VAS_on_arrival', n_patients)
    df['VAS_reduction'] = df['VAS_on_scene'] - df['VAS_on_arrival']
    str_df['VAS_reduction'] = get_continuous_str(df, 'VAS_reduction', n_patients)

    df['fentanyl_dose'] = 0
    df['ketamine_dose'] = 0
    df['esketamine_dose'] = 0
    df['morphine_dose'] = 0
    df['Alle Medikamente'] = df['Alle Medikamente'].str.replace(',', ';')
    for i, row in df.iterrows():
        if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
            continue
        for analgetic in row['Alle Medikamente'].split(';'):
            if analgetic.strip() == '':
                continue
            if '7IE' in analgetic:
                continue
            analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
            if 'Fentanyl' in analgetic and '/h' not in analgetic:
                dose = analgetic.split('Fentanyl')[-1].strip()
                df.at[i, 'fentanyl_dose'] += float(dose)
            elif 'Fentanyl' in analgetic and '/h' in analgetic:
                dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
                dose = float(dose) * df.at[i, 'mission_duration']
                df.at[i, 'fentanyl_dose'] += float(dose)
            elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
                dose = analgetic.split('Ketamin')[-1].strip()
                df.at[i, 'ketamine_dose'] += float(dose)
            elif 'Esketamin' in analgetic:
                dose = analgetic.split('Esketamin')[-1].strip()
                df.at[i, 'esketamine_dose'] += float(dose)
            elif 'Morphin' in analgetic or 'Morphine' in analgetic:
                dose = analgetic.split('Morphin')[-1].strip()
                df.at[i, 'morphine_dose'] += float(dose)

    df['fentanyl_given'] = df['fentanyl_dose'] > 0
    str_df['fentanyl'] = get_categorical_str(df, 'fentanyl_given', True, n_patients)
    str_df['fentanyl_dose'] = get_continuous_str(df, 'fentanyl_dose', n_patients)

    df['morphine_given'] = df['morphine_dose'] > 0
    str_df['morphine'] = get_categorical_str(df, 'morphine_given', True, n_patients)

    df['ketamine_given'] = df['ketamine_dose'] > 0
    df['esketamine_given'] = df['esketamine_dose'] > 0
    df['any_ketamine_given'] = df['ketamine_given'] | df['esketamine_given']
    str_df['ketamine'] = get_categorical_str(df, 'any_ketamine_given', True, n_patients)
    df['any_ketamine_dose'] = df['ketamine_dose'] + df['esketamine_dose']

    return str_df.T

In [ ]:
secondary_df = data_df.copy()
insufficient_df = secondary_df[secondary_df['VAS_on_arrival'] > 3]
sufficient_df = secondary_df[secondary_df['VAS_on_arrival'] <= 3]

secondary_table1 = table1(secondary_df)
sufficient_table1 = table1(sufficient_df)
insufficient_table1 = table1(insufficient_df)

secondary_t1 = pd.concat([secondary_table1, sufficient_table1, insufficient_table1], axis=1, keys=['Secondary', 'Sufficient', 'Insufficient'])
secondary_t1

In [ ]:
from pathlib import Path

output_dir = Path('/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/secondary_analgesia')
output_dir.mkdir(parents=True, exist_ok=True)

# Save final Table 1
if 'secondary_t1' in globals():
    try:
        secondary_t1.to_excel(output_dir / 'table1_secondary.xlsx')
    except Exception as exc:
        print(f'Could not save Table 1: {exc}')